In [ ]:
import csv
import os
from datetime import datetime
from googleapiclient.discovery import build

# 1. CONFIGURAÇÃO E INPUTS
api_key = "Insira_sua_chave_aqui"
youtube = build('youtube', 'v3', developerKey=api_key)

# Estratégia de Negócio: Monitorar a Líder vs. Concorrentes
# Usamos os 'Handles' (@) que são públicos e fáceis de achar.
canais_alvo = {
    "MK Music (Líder)": "@mkmusic",
    "Todah Music (Concorrente A)": "@TodahMusic",
    "Sony Music Gospel (Concorrente B)": "@SonyMusicGospel"
}

# Configuração de Pasta
base_usuario = os.path.expanduser("~")
pasta_destino = os.path.join(base_usuario, "Desktop")

# Verifica se o Desktop está no OneDrive
if not os.path.exists(pasta_destino):
    pasta_destino = os.path.join(base_usuario, "OneDrive", "Desktop")

# Se não achar nenhum, usa a pasta base
if not os.path.exists(pasta_destino):
    pasta_destino = base_usuario

caminho_arquivo = os.path.join(pasta_destino, "relatorio_gospel_completo.csv")
print(f"📍 O arquivo será salvo em: {caminho_arquivo}\n")
# 2. FUNÇÕES AUXILIARES (A MÁGICA ACONTECE AQUI)
def resolver_handle_para_id(handle):
    """
    Função avançada: Transforma o nome amigável (@mkmusic) no ID técnico (UC...).
    Isso evita que o engenheiro tenha que procurar IDs manualmente.
    """
    try:
        # Remove o @ se o usuário tiver colocado
        handle_limpo = handle.replace("@", "")
        
        # Busca pelo canal usando o handle (requisição de baixo custo: 1 unidade)
        res = youtube.search().list(
            part='snippet',
            q=handle,       # O termo de busca (ex: @mkmusic)
            type='channel', # Garante que só retorna canais
            maxResults=1    # Só queremos o primeiro/mais relevante
        ).execute()
        
        if res['items']:
            channel_id = res['items'][0]['snippet']['channelId']
            titulo_real = res['items'][0]['snippet']['channelTitle']
            print(f"   [OK] Handle '{handle}' resolvido para ID: {channel_id} ({titulo_real})")
            return channel_id
        else:
            print(f"   [ERRO] Canal não encontrado para o handle: {handle}")
            return None
            
    except Exception as e:
        print(f"   [ERRO CRÍTICO] Falha ao resolver handle {handle}: {e}")
        return None

def pegar_videos_recentes(channel_id, qtd=50):
    """
    Extrai os vídeos da playlist 'Uploads' (Todos os envios) do canal.
    """
    video_ids = []
    try:
        # 1. Descobrir a playlist de uploads desse canal
        res_canal = youtube.channels().list(
            id=channel_id, 
            part='contentDetails'
        ).execute()
        
        playlist_uploads = res_canal['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        
        # 2. Pegar os vídeos (Paginação simplificada para os últimos 50)
        res_videos = youtube.playlistItems().list(
            playlistId=playlist_uploads,
            part='contentDetails',
            maxResults=qtd 
        ).execute()
        
        for item in res_videos['items']:
            video_ids.append(item['contentDetails']['videoId'])
            
    except Exception as e:
        print(f"   [ERRO] Falha ao listar vídeos do canal {channel_id}: {e}")
        
    return video_ids

# 3. O PIPELINE DE EXECUÇÃO (ETL)
print(f"--- INICIANDO PIPELINE DE DADOS GOSPEL ---")
print(f"Data de Execução: {datetime.now().strftime('%d/%m/%Y')}\n")

# Prepara o arquivo CSV (Cabeçalho inteligente)
arquivo_novo = not os.path.isfile(caminho_arquivo)
with open(caminho_arquivo, mode='a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=';')
    
    # Se for arquivo novo, escreve o cabeçalho
    if arquivo_novo:
        writer.writerow(['Data_Extracao', 'Gravadora_Tag', 'Canal_ID', 'Video_ID', 'Titulo_Video', 'Views', 'Likes', 'Comentarios'])
        print(" Arquivo criado com sucesso! Iniciando coleta...\n")

    # LOOP PRINCIPAL: Para cada Gravadora no nosso Dicionário
    for tag_gravadora, handle in canais_alvo.items():
        print(f" Processando: {tag_gravadora}...")
        
        # PASSO 1: Descobrir o ID real (Resolução de Handle)
        id_tecnico = resolver_handle_para_id(handle)
        
        if id_tecnico:
            # PASSO 2: Listar os vídeos desse canal
            lista_videos = pegar_videos_recentes(id_tecnico, qtd=50) # Pegando 50 últimos
            print(f"   -> Extraindo métricas de {len(lista_videos)} vídeos recentes...")
            
            # PASSO 3: Coletar métricas detalhadas (Seu código original otimizado)
            videos_processados = 0
            for vid_id in lista_videos:
                try:
                    stats = youtube.videos().list(
                        part="snippet,statistics", 
                        id=vid_id
                    ).execute()
                    
                    if stats['items']:
                        item = stats['items'][0]
                        # Tratamento de dados (Transform)
                        titulo = item['snippet']['title']
                        views = item['statistics'].get('viewCount', 0)
                        likes = item['statistics'].get('likeCount', 0)
                        comments = item['statistics'].get('commentCount', 0)
                        data_hoje = datetime.now().strftime('%Y-%m-%d')
                        
                        # Carga (Load)
                        writer.writerow([data_hoje, tag_gravadora, id_tecnico, vid_id, titulo, views, likes, comments])
                        videos_processados += 1
                except Exception as e:
                    print(f"      Erro no vídeo {vid_id}: {e}")
            
            print(f"    {videos_processados} vídeos salvos com sucesso.\n")

print(f"--- FIM DO PIPELINE! Relatório disponível em: {caminho_arquivo} ---")